#### Import Json FIle to DF

In [58]:
pip install yellowbrick

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import os, glob
import os.path
from sklearn.cluster import KMeans 

In [2]:
df_lr = pd.read_json (r'linear_regression_klinikumbewertungen.json')
df_google_sent = pd.read_json (r'sentiment_comments_google.json')
df_kb_sent = pd.read_json (r'klinikbewertung_sentiment.json')
df_kb_comments_nr = pd.read_json (r'number_of_comments_klinikumbew.json')
df_google_comments_nr = pd.read_json (r'number_of_comments_google.json')

In [3]:
kliniklist = pd.read_excel(r'..\..\Klinikliste.xlsx')
df_kliniklist = pd.DataFrame(kliniklist, columns= ['Klinikname','Link Google Maps','Link Klinikbewertungen'])
df_kliniklist['klinik_ID'] = [i for i in range(1,25)]
df_kliniklist.reset_index(drop=True, inplace=True)

In [4]:
df_lr.head(1)

,Klinik_ID,Prediction,Klinikumbew_Gesamt,Accuracy
0,1,3.92,4.26,92.0


In [5]:
df_google_sent.head(1)

,Klinik_ID,Sentiment,GesamtsterneWithNull,GesamtsterneWithoutNull,Accuracy
0,1,3.79,4.29,4.29,88.28


In [6]:
df_google_comments_nr.head(1)

,Klinik_ID,Negative,Neutral,Positive,Total_Comments
0,1,9,24,74,107


In [7]:
df_kb_comments_nr.head(1)

,Klinik_ID,Negative,Neutral,Positive,Total_Comments
0,1,4,2,17,23


In [8]:
df_kb_sent["KB_Sentiment"] = df_kb_sent["KB_Sentiment"].round(2)

In [9]:
df_merge_kb = pd.concat([df_kb_sent.set_index('Klinik_ID'), df_lr.set_index('Klinik_ID'),
                      df_kb_comments_nr.set_index('Klinik_ID')], axis=1, join='inner').sort_values('Klinik_ID', ascending = True).reset_index()

In [11]:
df_merge_kb.head(2)

,Klinik_ID,KB_Sentiment,Prediction,Klinikumbew_Gesamt,Accuracy,Negative,Neutral,Positive,Total_Comments
0,1,3.77,3.92,4.26,92.00,4,2,17,23
1,2,4.03,4.64,4.80,96.61,6,7,57,70


In [12]:
df_merge_google = pd.concat([df_google_sent.set_index('Klinik_ID'), df_google_comments_nr.set_index('Klinik_ID')], 
                            axis=1, join='inner').sort_values('Klinik_ID', ascending = True).reset_index()

In [13]:
df_merge_google.head(2)

,Klinik_ID,Sentiment,GesamtsterneWithNull,GesamtsterneWithoutNull,Accuracy,Negative,Neutral,Positive,Total_Comments
0,1,3.79,4.29,4.29,88.28,9,24,74,107
1,2,3.36,3.86,3.67,91.72,9,38,39,86


#### Calculation of a SCHIFA SCORE

In [16]:
def schifa1(df1, c1, c2, c3, c4):
    calculate_point = ((df1[c1]*(2/3)+1) + (df1[c2]*(2/3)+1) + (df1[c3]*(2/3)+1)) * df1[c4]
    return calculate_point.round(2)

In [17]:
df_merge_kb['Total_Kb'] = schifa1(df_merge_kb, "KB_Sentiment", "Prediction", "Klinikumbew_Gesamt", "Total_Comments")

In [18]:
df_merge_kb.head(2)

,Klinik_ID,KB_Sentiment,Prediction,Klinikumbew_Gesamt,Accuracy,Negative,Neutral,Positive,Total_Comments,Total_Kb
0,1,3.77,3.92,4.26,92.00,4,2,17,23,252.23
1,2,4.03,4.64,4.80,96.61,6,7,57,70,838.60


In [19]:
def schifa2(df1, c1, c2, c3):
    calculate_point = (df1[c1] + df1[c2]) * df1[c3]
    return calculate_point.round(2)

In [20]:
df_merge_google['Total_Kb'] = schifa2(df_merge_google, "Sentiment", "GesamtsterneWithNull", "Total_Comments")

In [21]:
df_merge_google.head(2)

,Klinik_ID,Sentiment,GesamtsterneWithNull,GesamtsterneWithoutNull,Accuracy,Negative,Neutral,Positive,Total_Comments,Total_Kb
0,1,3.79,4.29,4.29,88.28,9,24,74,107,864.56
1,2,3.36,3.86,3.67,91.72,9,38,39,86,620.92


In [22]:
df_outer = pd.merge(df_merge_kb,df_merge_google,on='Klinik_ID',how='outer')

In [23]:
df_outer = df_outer.fillna(0)

In [24]:
df_outer.head(5)

,Klinik_ID,KB_Sentiment,Prediction,Klinikumbew_Gesamt,Accuracy_x,Negative_x,Neutral_x,Positive_x,Total_Comments_x,Total_Kb_x,Sentiment,GesamtsterneWithNull,GesamtsterneWithoutNull,Accuracy_y,Negative_y,Neutral_y,Positive_y,Total_Comments_y,Total_Kb_y
0,1,3.77,3.92,4.26,92.00,4,2,17,23,252.23,3.79,4.29,4.29,88.28,9.0,24.0,74.0,107.0,864.56
1,2,4.03,4.64,4.80,96.61,6,7,57,70,838.60,3.36,3.86,3.67,91.72,9.0,38.0,39.0,86.0,620.92
2,3,4.03,4.54,4.68,97.02,15,7,96,118,1396.33,3.65,4.21,4.21,86.74,10.0,47.0,81.0,138.0,1084.68
3,4,2.96,1.76,1.72,97.40,36,4,45,85,619.93,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00
4,5,3.60,3.20,3.11,97.10,38,9,157,204,1959.76,3.00,3.56,3.25,92.41,5.0,15.0,5.0,25.0,164.00


In [25]:
def schifa(df1, c1, c2, c3 , c4):
    calculate_point = (df1[c1] + df1[c2])/ (df1[c3]*3+df1[c4]*2)
    return calculate_point.round(2)

In [26]:
df_outer['SCHIFA'] = schifa(df_outer, "Total_Kb_x", "Total_Kb_y" , "Total_Comments_x", "Total_Comments_y")

#### Grouping

In [58]:
df_outer.loc[df_outer.SCHIFA>2.4, 'Shifa_Score']='Mangelhaft'
df_outer.loc[df_outer.SCHIFA>3.1, 'Shifa_Score']='Ausreichend'
df_outer.loc[df_outer.SCHIFA>3.4, 'Shifa_Score']='Gut'
df_outer.loc[df_outer.SCHIFA>3.6, 'Shifa_Score']='Sehr Gut'
df_outer.loc[df_outer.SCHIFA>3.9, 'Shifa_Score']='Hervorragend'

In [61]:
df_outer.head(5)

,Klinik_ID,KB_Sentiment,Prediction,Klinikumbew_Gesamt,Accuracy_x,Negative_x,Neutral_x,Positive_x,Total_Comments_x,Total_Kb_x,...,GesamtsterneWithNull,GesamtsterneWithoutNull,Accuracy_y,Negative_y,Neutral_y,Positive_y,Total_Comments_y,Total_Kb_y,SCHIFA,Shifa_Score
0,1,3.77,3.92,4.26,92.00,4,2,17,23,252.23,...,4.29,4.29,88.28,9.0,24.0,74.0,107.0,864.56,3.95,Hervorragend
1,2,4.03,4.64,4.80,96.61,6,7,57,70,838.60,...,3.86,3.67,91.72,9.0,38.0,39.0,86.0,620.92,3.82,Sehr Gut
2,3,4.03,4.54,4.68,97.02,15,7,96,118,1396.33,...,4.21,4.21,86.74,10.0,47.0,81.0,138.0,1084.68,3.94,Hervorragend
3,4,2.96,1.76,1.72,97.40,36,4,45,85,619.93,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,2.43,Mangelhaft
4,5,3.60,3.20,3.11,97.10,38,9,157,204,1959.76,...,3.56,3.25,92.41,5.0,15.0,5.0,25.0,164.00,3.21,Ausreichend


In [60]:
df_outer.to_json(r'SCHIFA.json')